# The analysis of multiscale model results: insights into a single file

This notebook shows various analyses of the results of the multiscale model. We go into the details of the pulse shaping, the plasma profile, and we show also the build up of the harmonic signal.

See complementary analyses:
* [Jupyter notebook processing a list of results.](./analyse_cell_list.ipynb)
* [Jupyter notebook analysing a file, which contains also the outputs of TDSE module](../density_profile/analyse_density_profile.ipynb) (these results were removed in the actual tutorial because they are the largest datasets from the model).



## Load libraries & initial data

In [ ]:
## python modules used within this notebook
import numpy as np
from scipy import integrate
from scipy import interpolate
import matplotlib.pyplot as plt
import matplotlib.animation
import matplotlib.colors as colors
import os
import h5py
import sys
import copy
import MMA_administration as MMA
import mynumerics as mn
import units
from IPython.display import display, Markdown
from IPython.display import HTML

import dataformat_CUPRAD as dfC
import HHG

anims_author = 'Jan Vábek'

matplotlib.rcParams['animation.embed_limit'] = 200.
# print(matplotlib.rcParams['animation.embed_limit'])


%matplotlib inline

Here we dicide if we `show` animations here or `save` them as well.

In [ ]:
visualisation = 'save'

## Load data

We load the data from the pulse propagation in the Pythonic data container defined [in this module](../../CUPRAD/python/dataformat_CUPRAD.py). It contains the data about the pulse propagation and some further characteristics. The data from the harmonic signal will be loaded later.

In [ ]:
demos_path = os.path.join(os.environ['MULTISCALE_DEMOS'],'gas_cell')

h5file1 = os.path.join(demos_path, 'results_cell_3.h5')

with h5py.File(h5file1,'r') as f:

    # load cuprad data = pulse propagation
    CUPRAD_res = dfC.get_data(f)
    CUPRAD_res.get_plasma(f)
    CUPRAD_res.get_plasma(f)
    CUPRAD_res.get_ionisation_model(f)
    

# Egrid = CUPRAD_res.ionisation_model.Egrid
# ionisation_rates = CUPRAD_res.ionisation_model.ionisation_rates

# ref_tables_file = os.path.join(os.environ['JUPYTER_EXAMPLES'],'gas_cell', MMA.filenames['ionisation_tables'])
# with h5py.File(ref_tables_file,'w') as f:
#     mn.adddataset(f ,'Egrid',Egrid,'[a.u.]')
#     mn.adddataset(f ,'ionisation_rates',ionisation_rates,'[a.u.]')


Here we print some basic characteristics of the simulation.

In [ ]:
# code to generate the following text
display(Markdown(
rf"""* The ($1/\mathrm{{e}}$) entry pulse duration ${
      CUPRAD_res.pulse_duration_entry   
      :.1f}~\mathrm{{fs}}$
* The box size is ($z-$ and $\rho$-grids start at 0):
    * $z_{{\mathrm{{max}}}}={
      1e3*CUPRAD_res.zgrid[-1]   
      :.1f}~\mathrm{{mm}},~N_z={
      np.shape(CUPRAD_res.E_zrt)[0]   
      :.0f}$,
    * $t_{{\mathrm{{min/max}}}}=\mp{
      1e15*CUPRAD_res.tgrid[-1]   
      :.0f}~\mathrm{{fs}},~N_t={
      np.shape(CUPRAD_res.E_zrt)[2]   
      :.0f}$,
    * $\rho_{{\mathrm{{max}}}}={
      1e6*CUPRAD_res.rgrid[-1]   
      :.0f}~\mu\mathrm{{m}},~N_\rho={
      np.shape(CUPRAD_res.E_zrt)[1]   
      :.0f}.$
"""))

## Plot the propagating pulse
We choose the time-and-space window to see the pulse as it propagates through the medium. Note that we measure the intensity by the "expected harmonic cutoff", these units are obtained by the formula $E_{\text{cut-off}} = I_P + 3.17U_p$ (it is directly proportional since $U_p$ is linearly proportional to the intensity). Aside we plot the plasma density (measured in % relative to the particle density).

There are more technical details about the data: We plot the pulse directly as it is stored in the file. This means that we a co-moving frame defined by the group velocity, $v_g$, of the pulse: this is the computational window of CUPRAD. The group velocity $v_g$ is defined from the linear dispersion relation and depends on the chosen reference pressure and central wavelength. Physically speaking, $v_g$ is arbitrary and needs to be considered in further processing. For example, the Pythonic class represented by `CUPRAD_res` contains methods to adjust to the reference given by the speed of light (both activelly by changing the data `CUPRAD_res.vacuum_shift()` or just by sychronising the clocks in the $t$-grid `CUPRAD_res.co_moving_t_grid(zgrid)`).$^\dagger$

$^\dagger$ Note that the shift `CUPRAD_res.vacuum_shift()` is retrieved as the Fourier shift, which imposes periodic conditions. It thus requires vanishing field close to boundaries and cannot be applied to the plasma density. `CUPRAD_res.co_moving_t_grid(zgrid)` is a function and it transforms any `zgrid`.

In [ ]:
tlim = np.asarray((-60,60))   # [fs]  the time range for plotting the propagating pulse
rlim = 300                    # [mum] the radial range for plotting the propagating pulse

frame_multiplier = 10 # stride in z for plotting

ani_outpath = os.path.join(os.environ['MULTISCALE_WORK_DIR'],'gas_cell', 'export')
if not(os.path.exists(ani_outpath)): os.makedirs(ani_outpath)

In [ ]:
# Code to generate the animated figure

k_t_min, k_t_max = mn.FindInterval(1e15*CUPRAD_res.tgrid,1.05*tlim)
k_r_max          = mn.FindInterval(1e6*CUPRAD_res.rgrid ,1.05*rlim)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 4))

r_grid, sym_data = mn.symmetrize_y(1e6*CUPRAD_res.rgrid[:k_r_max],
                    (
                    HHG.ComputeCutoff(
                        mn.FieldToIntensitySI(CUPRAD_res.E_zrt[0,:k_r_max,k_t_min:k_t_max])/units.INTENSITYau,
                        mn.ConvertPhoton(CUPRAD_res.omega0,'omegaSI','omegaau'),
                        mn.ConvertPhoton(CUPRAD_res.Ip_eV,'eV','omegaau')
                    )[1]
                    ).T)

pc1 = ax1.pcolormesh(1e15*CUPRAD_res.tgrid[k_t_min:k_t_max], r_grid, sym_data.T, shading='auto')

ax1.set_xlim(tlim)
ax1.set_ylim((-rlim,rlim))

ax1.set_title("z={:.2f}".format(1e3*CUPRAD_res.zgrid[0]) + ' mm')
ax1.set_xlabel(r'$t~[\mathrm{fs}]$')
ax1.set_ylabel(r'$\rho~[\mu\mathrm{m}]$')

cbar1 = fig.colorbar(pc1, ax=ax1)
cbar1.ax.set_ylabel(r'Intensity [harmonic cut-off]', rotation=90)

r_grid, sym_data = mn.symmetrize_y(1e6*CUPRAD_res.plasma.rgrid[:k_r_max],
                         (1e2/CUPRAD_res.effective_neutral_particle_density)*(CUPRAD_res.plasma.value_zrt[0,:k_r_max,k_t_min:k_t_max]).T
                                    )

pc2 = ax2.pcolormesh(1e15*CUPRAD_res.plasma.tgrid[k_t_min:k_t_max], r_grid, sym_data.T, shading='auto')

cbar2 = fig.colorbar(pc2, ax=ax2)
cbar2.ax.set_ylabel(r'relative plasma density [%]', rotation=90)

ax2.set_xlabel(r'$t~[\mathrm{fs}]$')
ax2.set_ylabel(r'$\rho~[\mu\mathrm{m}]$')


def update(frame):
    # Update the data
    data = (mn.symmetrize_y(1e6*CUPRAD_res.rgrid[:k_r_max], (
            HHG.ComputeCutoff(
                        mn.FieldToIntensitySI(CUPRAD_res.E_zrt[frame_multiplier*frame,:k_r_max,k_t_min:k_t_max])/units.INTENSITYau,
                        mn.ConvertPhoton(CUPRAD_res.omega0,'omegaSI','omegaau'),
                        mn.ConvertPhoton(CUPRAD_res.Ip_eV,'eV','omegaau')
                    )[1]        
            ).T)[1]).T
    
    # Update the colors
    pc1.set_array(data.ravel())
    pc1.set_clim(data.min(), data.max())
    cbar1.update_normal(pc1)

    ax1.set_title("z={:.2f}".format(1e3*CUPRAD_res.zgrid[frame_multiplier*frame]) + ' mm')

    data = (mn.symmetrize_y(1e6*CUPRAD_res.plasma.rgrid[:k_r_max],
                          (1e2/CUPRAD_res.effective_neutral_particle_density)*(CUPRAD_res.plasma.value_zrt[frame_multiplier*frame,:k_r_max,k_t_min:k_t_max]).T
                                    )[1]).T
    
    pc2.set_array(data.ravel())
    pc2.set_clim(data.min(), data.max())
    cbar2.update_normal(pc2)

    return [pc1, pc2]

# Ensure the layout does not have overlaps and everything is nicely spaced
fig.tight_layout() 

ani = matplotlib.animation.FuncAnimation(fig, update, frames=len(CUPRAD_res.zgrid)//frame_multiplier, blit=True)
# ani = matplotlib.animation.FuncAnimation(fig, update, frames=10, blit=True)


if visualisation == 'save':
    # Define the writer using ffmpeg for mp4 format and save it
    Writer = matplotlib.animation.writers['ffmpeg']
    writer = Writer(fps=30, metadata=dict(artist=anims_author), bitrate=1800)

    ani.save(os.path.join(ani_outpath,'gas_cell_single_pulse2.mp4'), writer=writer)

plt.close(fig)

HTML(ani.to_jshtml())


## XUV camera

Here we show the far-field XUV spectra together with the build-up of the signal in the generating medium.$^\dagger$

These data might be very large. Please specify here the parametes of the plot, so only the necessary subset of data is loaded and processed.

$^\dagger$ Note that the maximum of the harmonic signal is obttained simply as $I_H(z) = \mathrm{max}_{\omega_0 \xi \in \left[H-\Delta, H+\Delta H \right]}|\mathcal{E}_{\text{XUV, cumulative}}(z,\cdot,\xi)|^2$, we define below $\Delta H$=`delta_H`. One might consider also other metrics as the total energy, etc.

In [ ]:
rmax = 0.007                # [m]    the radial dimesion to read the data
XUV_theta_range = [-2, 2]   # [mrad] the divergence angle for plotting 
orders_to_plot = 3          # the range of the logarithmic plot of the spatially resolved harmonic spectra
# Hmax_plot_linear = 50       # the maximal frequency in the linear plot of the spatially resolved harmonic spectra
Hmax_plot          = 36


kz_step = 10    #       the step in $z$ for plotting (derived from the spacing used in the computational grid)

H_interest = np.asarray([19, 25, 37, 49])   # harmonics for which we show the build-up
multipliers = [1,4,10,150]                  # multipliers applied to the build-up to fit the figure nicely
delta_H = 1.                                # the camera spectral range to analyse signal of the harmoonics of the interest

In [ ]:
# import data & basic analyses
with h5py.File(h5file1,'r') as f1:
    # load Hankel data = XUV camera
    ogrid_Hankel = f1[MMA.paths['Hankel_outputs']+'/ogrid'][:]
    rgrid_Hankel = f1[MMA.paths['Hankel_outputs']+'/rgrid'][:]
    zgrid_Hankel = f1[MMA.paths['Hankel_outputs']+'/zgrid'][0:-1:kz_step]

    camera_distance = mn.readscalardataset(f1,MMA.paths['Hankel_inputs']+'/distance_FF','N')
    theta_grid_Hankel = np.arctan(rgrid_Hankel/camera_distance)  # recompute the radial grid to the divergence

    Hgrid_Hankel = ogrid_Hankel/CUPRAD_res.omega0

    kr_max = mn.FindInterval(rgrid_Hankel, rmax) + 1
    rgrid_Hankel_full = copy.copy(rgrid_Hankel)
    rgrid_Hankel = rgrid_Hankel[:kr_max]
    theta_grid_Hankel = theta_grid_Hankel[:kr_max]


    cumulative_field =    f1[MMA.paths['Hankel_outputs']+'/cumulative_field'][0:-1:kz_step,:kr_max,:,0] +\
                    1j*f1[MMA.paths['Hankel_outputs']+'/cumulative_field'][0:-1:kz_step,:kr_max,:,1] 
    
    entry_plane_transform = f1[MMA.paths['Hankel_outputs']+'/entry_plane_transform'][:,:,0] +\
                    1j*f1[MMA.paths['Hankel_outputs']+'/entry_plane_transform'][:,:,1]

    exit_plane_transform = f1[MMA.paths['Hankel_outputs']+'/exit_plane_transform'][:,:,0] +\
                1j*f1[MMA.paths['Hankel_outputs']+'/exit_plane_transform'][:,:,1]

# find maxima of the harmonics of the interest
H_idx = [tuple(mn.FindInterval(Hgrid_Hankel,(H_interest[k1]-delta_H, H_interest[k1]+delta_H))) for k1 in range(len(H_interest))]
H_max_interest = [np.max(np.abs(cumulative_field[:,:,H_idx[k1][0]:H_idx[k1][1]]),axis=(1,2)) for k1 in range(len(H_interest))]    

In [ ]:
# Code to create the following animated figure
fig = plt.figure(figsize=(14, 6))

# Define subplots using subplot2grid
ax1 = plt.subplot2grid((3, 2), (0, 0), rowspan=2)  # Upper left
ax2 = plt.subplot2grid((3, 2), (0, 1), rowspan=2)  # Upper right
ax3 = plt.subplot2grid((3, 2), (2, 0), colspan=2)  # Bottom, spanning both columns

theta_grid_Hankel = theta_grid_Hankel[:kr_max]
r_grid, sym_data = mn.symmetrize_y(rgrid_Hankel, np.abs(cumulative_field[0,:,:]).T)
theta_grid_sym, sym_data = mn.symmetrize_y(theta_grid_Hankel, np.abs(cumulative_field[0,:,:]).T)



pc1 = ax1.pcolormesh(Hgrid_Hankel, 1e3*theta_grid_sym, sym_data.T, shading='auto')
pc2 = ax2.pcolormesh(Hgrid_Hankel, 1e3*theta_grid_sym, sym_data.T, shading='auto',norm=colors.LogNorm(vmin=(10**(-orders_to_plot))*sym_data.max(), vmax=sym_data.max()))

ax1.set_ylim(XUV_theta_range)
ax2.set_ylim(XUV_theta_range)

# ax1.set_xlim(Hgrid_Hankel[0],Hmax_plot_linear)

ax1.set_title('spatially resolved XUV spectrum (linear scale)')
ax2.set_title('spatially resolved XUV spectrum (log scale)')
ax1.set_ylabel(r'divergence [mrad]')
for cbar in (cbar1, cbar2): cbar.ax.set_xlabel(r'$|\mathcal{E}_{XUV}|$ [arb.u.]')
ax1.set_ylabel(r'divergence [mrad]')
H_min, H_max = ax1.get_xlim() 
odd_ticks_major = range(int(np.ceil(H_min)) | 1, int(np.floor(H_max)) + 1, 10) # Generate major odd ticks within the H-range
odd_ticks_minor = range(int(np.ceil(H_min)) | 1, int(np.floor(H_max)) + 1, 2)  # Generate odd ticks within the H-range
for ax in (ax1, ax2):
    ax.set_xlabel('harmonic order [-]')
    ax.set_xticks(odd_ticks_major)
    ax.set_xticks(odd_ticks_minor, minor = True)


cbar1 = fig.colorbar(pc1, ax=ax1)
cbar2 = fig.colorbar(pc2, ax=ax2) #, orientation='horizontal')
cbar2.ax.set_ylabel(r'$|\mathcal{E}_{XUV}|$ [arb.u.]', rotation=90)

# plot lines at selected harmonic orders
for k1 in range(len(H_interest)):
    ax1.plot(2*[H_interest[k1]],
             1e3*theta_grid_sym[-1]*np.asarray([-1,1]),
             'w:',alpha = 0.4)
    ax2.plot(2*[H_interest[k1]],
             1e3*theta_grid_sym[-1]*np.asarray([-1,1]),
             'w:',alpha = 0.4)



title = fig.suptitle("z={:.2f}".format(1e3*CUPRAD_res.zgrid[0]) + ' mm')
ax3.set_xlabel(r'$z~[\mathrm{mm}]$')

progress_line, = ax3.plot([], [], 'r-')  

ax3.set_xlim((1e3*CUPRAD_res.zgrid[0],1e3*CUPRAD_res.zgrid[-1]))

# normalised signals
max_signal = np.max([signal for signal in H_max_interest])
# multipliers = [max_signal/np.max(signal) for signal in H_max_interest]

for k1 in range(len(H_interest)):
    if (len(zgrid_Hankel) == len(H_max_interest[k1][:])): signal_plot = H_max_interest[k1][:]
    else: signal_plot = np.append(0,H_max_interest[k1][:])
    ax3.plot(1e3*zgrid_Hankel,multipliers[k1]*signal_plot, label='H'+str(H_interest[k1])+f' (x {multipliers[k1]:.1f})')

ax3.set_ylabel(r'XUV signal $[\mathrm{arb. u.}]$')
ax3.legend()

def update(frame):
    # Update the data
    data = (mn.symmetrize_y(rgrid_Hankel, np.abs(cumulative_field[frame,:,:]).T)[1]).T

    pc1.set_array(data.ravel())
    pc1.set_clim(data.min(), data.max())

    pc2.set_array(data.ravel())
    pc2.set_clim((10**(-orders_to_plot))*data.max(), data.max())

    title.set_text("z={:.2f}".format(1e3*zgrid_Hankel[frame+1]) + ' mm')

    # Update the progress indicator
    progress_line.set_data([1e3*zgrid_Hankel[frame+1], 1e3*zgrid_Hankel[frame+1]],
                             ax3.get_ylim())

    return [pc1,pc2, progress_line]


# Ensure the layout does not have overlaps and everything is nicely spaced
fig.tight_layout()

ani = matplotlib.animation.FuncAnimation(fig, update, frames=len(zgrid_Hankel)-1, blit=True)
# ani = matplotlib.animation.FuncAnimation(fig, update, frames=3, blit=True) # for testing the plot


if visualisation == 'save':
    # Define the writer using ffmpeg for mp4 format and save it
    Writer = matplotlib.animation.writers['ffmpeg']
    writer = Writer(fps=3, metadata=dict(artist=anims_author), bitrate=1800)

    ani.save(os.path.join(ani_outpath,'gas_cell_single_spectrum2.mp4'), writer=writer)

plt.close(fig)
HTML(ani.to_jshtml())


The output data also contain the Hankel transforms of the sources at the entry and exit plane of the medium.

In [ ]:
# Code to generate the figures of the Hankel transforms
logscale = True
normalise_spectra = True
orders_to_plot = 3 


fig = plt.figure(figsize=(17, 7))

# Define subplots using subplot2grid
ax1 = plt.subplot2grid((1, 2), (0, 0))  # Upper left
ax2 = plt.subplot2grid((1, 2), (0, 1))  # Upper right

theta_grid_Hankel = np.arctan(rgrid_Hankel_full/camera_distance)  # recompute the radial grid to the divergence
theta_grid_sym, sym_data = mn.symmetrize_y(theta_grid_Hankel, np.abs(entry_plane_transform).T/np.max(np.abs(entry_plane_transform)) if normalise_spectra
                                                              else np.abs(entry_plane_transform).T)
scale_kwargs = {'norm' : colors.LogNorm(vmin=(10**(-orders_to_plot))*sym_data.max(), vmax=sym_data.max())} if logscale else {}
pc1 = ax1.pcolormesh(Hgrid_Hankel, 1e3*theta_grid_sym, sym_data.T, shading='auto',**scale_kwargs)
cbar1 = fig.colorbar(pc1, ax=ax1, orientation = 'horizontal')


theta_grid_sym, sym_data = mn.symmetrize_y(theta_grid_Hankel, np.abs(exit_plane_transform).T/np.max(np.abs(exit_plane_transform)) if normalise_spectra
                                                              else np.abs(exit_plane_transform).T)
scale_kwargs = {'norm' : colors.LogNorm(vmin=(10**(-orders_to_plot))*sym_data.max(), vmax=sym_data.max())} if logscale else {}
pc2 = ax2.pcolormesh(Hgrid_Hankel, 1e3*theta_grid_sym, sym_data.T, shading='auto',**scale_kwargs)
cbar2 = fig.colorbar(pc2, ax=ax2, orientation = 'horizontal')

for cbar in (cbar1, cbar2): cbar.ax.set_xlabel(r'$|\mathcal{E}_{XUV}|$ [arb.u.]')
ax1.set_ylabel(r'divergence [mrad]')
H_min, H_max = ax1.get_xlim() 
odd_ticks_major = range(int(np.ceil(H_min)) | 1, int(np.floor(H_max)) + 1, 10) # Generate odd ticks within the H-range
odd_ticks_minor = range(int(np.ceil(H_min)) | 1, int(np.floor(H_max)) + 1, 2)  # Generate odd ticks within the H-range
for ax in (ax1, ax2):
    ax.set_xlabel('harmonic order [-]')
    ax.set_xticks(odd_ticks_major)
    ax.set_xticks(odd_ticks_minor, minor = True)

scale_text = '(log scale)' if logscale else '(linear scale)'
ax1.set_title(r'entrance-plane Hankel transform '+scale_text)
ax2.set_title(r'exit-plane Hankel transform '+scale_text)

plt.show()